# Drone Images Segmentation Using SegNet

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.transforms import v2
from torchvision import tv_tensors
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import InterpolationMode

from sklearn.model_selection import train_test_split

import os
import time
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt


# Dataset Class

In [ ]:
class DroneImagesSegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, filenames, spatial_transforms=None, normalize=None):
        self.images_dir = images_dir
        self.masks_dir = images_dir
        self.filenames = filenames
        self.spatial_transforms = spatial_transforms
        self.normalize = normalize

    def __len__(self):
        return len(self.images_dir)
    
    def __getitem__(self, index):
        image_path = os.path.join(self.images_dir, self.filenames[index])
        mask_path = os.path.join(self.masks_dir, self.filenames[index])
        
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("RGB")

        image = tv_tensors(image)
        mask = tv_tensors(mask)

        if self.spatial_transforms:
            image, mask = self.spatial_transforms(image, mask)
        
        if self.normalize:
            image = self.normalize(image)
        
        mask = (mask > 0).to(torch.float32)

        return image, mask

# Helper Functions

In [ ]:
def get_transforms(is_train=True):
    
    if is_train: 
        # Joint Spatial Transforms
        joint_transforms = v2.Compose([
            v2.Resize((256, 256), interpolation=InterpolationMode.NEAREST),
            
            v2.RandomHorizontalFlip(p=0.5),
            v2.RandomVerticalFlip(p=0.5),

            v2.RandomRotation(
                degrees=45,
                interpolation=InterpolationMode.NEAREST
            ),

            v2.RandomResizedCrop(
                size=(256, 256),
                scale=(0.8, 1.0),
                interpolation=InterpolationMode.NEAREST
            ),
            
            v2.ToImage(), # Image to tensor (.ToTensor() Alternative)
            v2.ToDtype(torch.float32, scale=True)
        ])

        # Image Only Transforms
        image_transforms = v2.Compose([
            v2.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2
            ),
            v2.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    else:
        joint_transforms = v2.Compose([
            v2.Resize((256, 256), interpolation=InterpolationMode.NEAREST),
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
        ])

        image_transforms = v2.Compose([
            v2.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    
    return joint_transforms, image_transforms

# Model Architecture